In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import Normalizer , scale
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error, r2_score,mean_absolute_error 
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from sklearn.utils import shuffle
from keras.utils import plot_model
import pickle
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
print("Loading checkins")
checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkins = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\Den_checkins.csv', sep=',', names=checkin_cols, encoding='latin-1').dropna(axis=1)

print("Loading POIs")
venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\Den_pois.csv', sep=',', names=venue_cols, encoding='latin-1')

#One checkin for each user
users = checkins.copy()
users.drop_duplicates(subset="user_id", keep = 'first', inplace = True)
print("Checkins: ", len(checkins))
print("Users: ", len(users))
len_checkins = len(checkins)
len_users = len(users)

#The rest of the checkins and categories
checkins_rest = users.merge(checkins, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']
print("Gotten: ", len(checkins_rest))

#One of each category in checkins_rest
categories1 = pd.DataFrame(pois, columns=['poi_id', 'category'])
categories1 = checkins_rest.merge(categories1, on='poi_id')
users_categories1 = categories1.copy()
users_categories1.drop_duplicates(subset="category", keep = 'first', inplace = True)
print("Unique categories in checkins_rest: ", len(users_categories1))

#FINISHING NOTE: Vi vil have alle colplement af checkins_rest, når vi sammenligner med categories_cat_no_cat, så har vi de 3 gange checkins vi har brug for. Så kan vi lave train/test på det store af dem.
categories_cat_no_cat = pd.DataFrame(users_categories1, columns=['user_id', 'poi_id', 'timestamp', 'timezone', 'category'])
checkins_rest_no_merge = pd.DataFrame(checkins_rest, columns=['user_id', 'poi_id', 'timestamp', 'timezone'])
poisandcategories = pd.DataFrame(pois, columns=['poi_id', 'category'])
checkins_rest_no_merge = checkins_rest_no_merge.merge(poisandcategories, on='poi_id')
cat_length = len(categories_cat_no_cat)
print("Total categories: ", len(categories_cat_no_cat))

test = categories_cat_no_cat.merge(checkins_rest_no_merge, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']
#test = pd.DataFrame(checkins_rest, columns=['user_id', 'poi_id', 'timestamp', 'timezone'])
#cattest = pd.DataFrame(pois, columns=['poi_id', 'category'])
#test.merge(cattest, on='poi_id')
#checkins_rest = cattest.merge(test, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']

restset = pd.DataFrame(test, columns=['user_id', 'poi_id', 'timestamp', 'timezone'])
userset = users
categoryset = pd.DataFrame(users_categories1, columns=['user_id', 'poi_id', 'timestamp', 'timezone'])

print("Overview:")
print(userset)
print(categoryset)
print(restset)

encoding = pd.DataFrame(userset, columns=['user_id'])
#print(encoding)
encoding_array = {}
temp = 0
for user in encoding.iterrows():
    user = user[0]
    value = encoding._get_value(user, 'user_id')
    encoding_array[value] = temp
    temp += 1

In [ ]:
print(encoding_array)

In [121]:
np.save(r"C:\Users\lasse\Desktop\RecommenderDL\datasets\d1.npy", encoding_array)
d2=np.load(r"C:\Users\lasse\Desktop\RecommenderDL\datasets\d1.npy", allow_pickle=True)
test = dict(enumerate(d2.flatten(), 1))[1]

In [120]:
print(checkins)

          user_id                    poi_id                       timestamp  \
524329     233919  4c2f45da66e40f47aa8ec18b  Tue Apr 03 18:19:44 +0000 2012   
731652     190585  4adcdb00f964a520775e21e3  Tue Apr 03 18:30:09 +0000 2012   
731653      24779  4adcdb00f964a520775e21e3  Sun Apr 29 11:33:55 +0000 2012   
731654      30835  4adcdb00f964a520775e21e3  Wed Jun 27 14:49:26 +0000 2012   
731655       3884  4adcdb00f964a520775e21e3  Mon Nov 12 11:53:23 +0000 2012   
...           ...                       ...                             ...   
33257607    45339  4ae1d14ef964a520ac8721e3  Sat Sep 14 20:42:29 +0000 2013   
33257719   241203  5234d16f11d2aa27377c1d6d  Sat Sep 14 21:14:11 +0000 2013   
33261149   201974  4c4ec7deea24c9b6e7143a16  Mon Sep 16 09:34:09 +0000 2013   
33261538    44648  4adcdafcf964a5205a5d21e3  Mon Sep 16 10:50:16 +0000 2013   
33262742   216973  4b48e2c6f964a520be5b26e3  Mon Sep 16 16:52:20 +0000 2013   

          timezone  
524329         120  
731652   

In [123]:
print(test.get(70246))

910


In [2]:
class EmbModel(tf.keras.Model):
    def __init__(self, useridlength, category_length):
        super(EmbModel, self).__init__()
        self.d_steps = 1
        self.useridlength = useridlength
        self.category_length = category_length
        self.model = self.init_model()
        print(self.useridlength)
        
    def call(self, inputs):
        return
    
    def init_model(self):
        poi_latitude_input = keras.layers.Input(shape=(1,), name='poi_latitude')
        poi_longitude_input = keras.layers.Input(shape=(1,), name='poi_longitude')
        poi_concat_input = tf.keras.layers.Concatenate(axis=-1)([poi_latitude_input, poi_longitude_input])
        #input_length:  #This is the length of input sequences, as you would define for any input layer of a Keras model. 
                        #For example, if all of your input documents are comprised of 1000 words, this would be 1000
        #input_dim: 
                        #This is the size of the vocabulary in the text data. 
                        #For example, if your data is integer encoded to values between 0-10, then the size of the vocabulary would be 11 words.
        poi_dense = keras.layers.Dense(8)(poi_concat_input)
        poi_reshape = keras.layers.Reshape((1, 8))(poi_dense)
        
        category_input = keras.layers.Input(shape=(1), name='category_input')
        category_emb = keras.layers.Embedding(self.category_length, 8)(category_input)    
        category_concat = tf.keras.layers.Concatenate(axis=-1)([category_emb, poi_reshape])
    
        user_input = keras.layers.Input(shape=(1,), name='user_id')
        user_emb = keras.layers.Embedding(self.useridlength, 16)(user_input)
        #user_reshape = layers.Reshape((1, 256))(user_emb)
                                    
        dot = keras.layers.Dot(axes=(2))([category_concat, user_emb])
            
        model = Model([category_input, poi_latitude_input, poi_longitude_input, user_input], dot)
        model.summary()
        return model
    
    def compile_model(self, optimizer):
        super(EmbModel, self).compile(run_eagerly=True)
        self.optimizer = optimizer
        
    def train_step(self, data):
        if len(data) == 3:
            real_data, labels, sample_weight = data
        else:
            sample_weight = None
            real_data, labels = data
        cat_data = real_data[0]
        lat_data = real_data[1]
        long_data = real_data[2]
        user_data = real_data[3]

        for i in range(self.d_steps):
            with tf.GradientTape() as tape:
                
                #print(latlong_data[0])
                #print(latlong_data[1])
                #print(user_data)
                
                dotproduct = self.model([cat_data, lat_data, long_data, user_data])
                #print(dotproduct)
                # Loss function = ||S-GroundTruth|| 
                loss = tf.math.abs(tf.subtract(tf.cast(dotproduct, tf.float64), labels))
                #print(loss)
            d_gradient = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(d_gradient, self.model.trainable_variables))
        return {'loss': loss}
    
    def predict_step(self, data):
        sample_weight = None
        cat_data = data[0]
        lat_data = data[1]
        long_data = data[2]
        user_data = data[3]
        return self.model([cat_data, lat_data, long_data, user_data])
    
    def load_model(self, path):
        self.model = keras.models.load_model(path)

In [3]:
model = EmbModel(1, 1)

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
poi_latitude (InputLayer)       [(None, 1)]          0                                            
__________________________________________________________________________________________________
poi_longitude (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 2)            0           poi_latitude[0][0]               
                                                                 poi_longitude[0][0]              
__________________________________________________________________________________________________
category_input (InputLayer)     [(None, 1)]          0                                 

In [109]:
model = keras.models.load_model(r'C:\Users\lasse\Desktop\RecommenderDL\models\model512')
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.0, beta_2=0.99, epsilon=1e-8)
model.compile(
    optimizer
)

In [110]:
x_test_df = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\y_train_df_512.csv', sep=',', names=['User','Latitude','Longitude', '0'], encoding='latin-1')[1:]

y_test_df = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\y_test_df_512.csv', sep=',', names=['0'], encoding='latin-1')[1:]

C:\Users\lasse\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (1,2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [111]:
encoding_array = np.load(r"C:\Users\lasse\Desktop\RecommenderDL\datasets\encoding_array_512.npy", allow_pickle=True)
encoding_array = dict(enumerate(encoding_array.flatten(), 1))[1]

In [ ]:
truePositive = 0
falsePositive = 0
trueNegative = 0
falseNegative = 0
change = 0
scores = []

for index, row in x_test_df.iterrows():
    long = float(x_test_df.loc[index, 'Longitude'])
    lat = float(x_test_df.loc[index, 'Latitude'])
    user = float(x_test_df.loc[index, 'User'])
    user = encoding_array.get(user)
    cat = float(x_test_df.loc[index, '0'])
    groundtruth = y_test_df.loc[index, '0']

    if visited == 1:
            print(score)

    long = tf.convert_to_tensor([long],dtype=None, dtype_hint=None, name=None)
    lat = tf.convert_to_tensor([lat],dtype=None, dtype_hint=None, name=None)
    user = tf.convert_to_tensor([user],dtype=None, dtype_hint=None, name=None)
    cat = tf.convert_to_tensor([cat],dtype=None, dtype_hint=None, name=None)

    score = model([cat, lat, long, user])
    print(score)
    
    if score > 0.01:
        prediction = 1
    else:
        prediction = 0

    if prediction == 1 and groundtruth == 1:
        truePositive += 1
    elif prediction == 1 and groundtruth == 0:
        falsePositive += 1
    elif prediction == 0 and groundtruth == 0:
        trueNegative += 1
    elif prediction == 0 and groundtruth == 1:
        falseNegative += 1
print("______________________________________________")
print("Results:")
print("True positive: ", truePositive)
print("False positive: ", falsePositive)
print("True negative: ", trueNegative)
print("False negative: ", falseNegative)
accuracy = (truePositive + trueNegative) / (truePositive + trueNegative + falsePositive + falseNegative)
precision = truePositive / (truePositive + falsePositive)
recall = truePositive / (truePositive + falseNegative)
f1 = 2 * ((precision * recall) / (precision + recall))

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)
print("_______________________________________________")

yo


In [ ]:
    test = checkin_data.loc[
    (checkin_data['user_id'] == x_test_df.loc[index, 'User']) & 
    (checkin_data['latitude'] == x_test_df.loc[index, 'Latitude']) & 
    (checkin_data['longitude'] == x_test_df.loc[index, 'Longitude'])]
    
    long = tf.convert_to_tensor([row['Longitude']],dtype=None, dtype_hint=None, name=None)
    lat = tf.convert_to_tensor([row['Latitude']],dtype=None, dtype_hint=None, name=None)
    user = tf.convert_to_tensor([row['User']],dtype=None, dtype_hint=None, name=None)
    score = model.predict([cat, lat, long, user])

In [ ]:
x_test_df = pd.DataFrame(x_test, columns=['User','Latitude','Longitude', '0'])
y_test_df = pd.DataFrame(y_test)

In [32]:
print("Loading checkins")
checkin_cols = ['user_id', 'poi_id', 'timestamp', 'timezone']
checkins = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\Den_checkins.csv', sep=',', names=checkin_cols, encoding='latin-1').dropna(axis=1)

print("Loading POIs")
venue_cols = ['poi_id', 'latitude', 'longitude', 'category', 'country_code']
pois = pd.read_csv(r'C:\Users\lasse\Desktop\RecommenderDL\datasets\Den_pois.csv', sep=',', names=venue_cols, encoding='latin-1')

c = pd.DataFrame(checkins, columns=['user_id', 'poi_id'])
p = pd.DataFrame(pois, columns=['poi_id', 'latitude', 'longitude', 'category'])

categories1 = p.merge(c, on='poi_id')

Loading checkins
Loading POIs


In [99]:
print(x_test_df)

              User   Latitude  Longitude     0
1.0       140208.0  55.778178  12.498369  30.0
2.0       167915.0  55.661986  12.610502  33.0
3.0       215805.0  55.678844  12.571694   5.0
4.0       266428.0  55.757718  12.516561   5.0
5.0       156019.0   55.67946  12.575359  87.0
...            ...        ...        ...   ...
328779.0   38574.0  55.882216  12.497458  11.0
328780.0    5523.0   55.88127  12.498967   1.0
328781.0  189543.0  55.668259  12.557653  12.0
328782.0  201860.0  55.674305  12.559395   8.0
328783.0  201337.0  55.688424  12.504158  30.0

[328783 rows x 4 columns]


In [107]:
for index, row in x_test_df.iterrows():
    long = float(x_test_df.loc[index, 'Longitude'])
    lat = float(x_test_df.loc[index, 'Latitude'])
    user = float(x_test_df.loc[index, 'User'])
    cat = float(x_test_df.loc[index, '0'])
    groundtruth = y_test_df.loc[index, '0']
    
    if groundtruth == 0:
        test = categories1.loc[(categories1['user_id'] == user) & (categories1['latitude'] == lat) & (categories1['longitude'] == long)]
        boolean = test.empty
        if not boolean:
            print("ERROR")
            print(index)
            print(user)
            print(lat)
            print(long)
            print(groundtruth)
            user = encoding_array.get(user)
            long = tf.convert_to_tensor([long],dtype=None, dtype_hint=None, name=None)
            lat = tf.convert_to_tensor([lat],dtype=None, dtype_hint=None, name=None)
            user = tf.convert_to_tensor([user],dtype=None, dtype_hint=None, name=None)
            cat = tf.convert_to_tensor([cat],dtype=None, dtype_hint=None, name=None)
            score = model([cat, lat, long, user])
            print(score)
    elif groundtruth == 1:
        test = categories1.loc[(categories1['user_id'] == user) & (categories1['latitude'] == lat) & (categories1['longitude'] == long)]
        boolean = test.empty
        if boolean:
            print("ERROR")
            print(index)
            print(user)
            print(lat)
            print(long)
            print(groundtruth)

ERROR
2839.0
24356.0
55.628444
12.649469
0.0
tf.Tensor([[[-0.02872161]]], shape=(1, 1, 1), dtype=float32)
ERROR
5905.0
83051.0
55.628444
12.649469
0.0
tf.Tensor([[[-0.01290249]]], shape=(1, 1, 1), dtype=float32)
ERROR
7021.0
9912.0
55.661622
12.604347
0.0
tf.Tensor([[[-0.00159838]]], shape=(1, 1, 1), dtype=float32)
ERROR
7585.0
155807.0
55.68315
12.571106
0.0
tf.Tensor([[[0.0085336]]], shape=(1, 1, 1), dtype=float32)
ERROR
9631.0
21859.0
55.680156
12.582083
0.0
tf.Tensor([[[-0.01703239]]], shape=(1, 1, 1), dtype=float32)
ERROR
12146.0
9912.0
55.661622
12.604347
0.0
tf.Tensor([[[-0.00159838]]], shape=(1, 1, 1), dtype=float32)
ERROR
13954.0
18933.0
55.628444
12.649469
0.0
tf.Tensor([[[-0.03127731]]], shape=(1, 1, 1), dtype=float32)
ERROR
16807.0
38850.0
55.672043
12.565012
0.0
tf.Tensor([[[-0.01223923]]], shape=(1, 1, 1), dtype=float32)
ERROR
18105.0
75696.0
55.67256
12.565074
0.0
tf.Tensor([[[0.02150839]]], shape=(1, 1, 1), dtype=float32)
ERROR
19765.0
188647.0
55.628444
12.649469
0.0
t

KeyboardInterrupt: 

In [119]:
259108.0
55.737798
12.522954
test = categories1.loc[(categories1['user_id'] == 71191.0) & (categories1['latitude'] == 55.737798) & (categories1['longitude'] == 12.522954)]
print(test)

                         poi_id   latitude  longitude        category  user_id
10445  51973b01498e76a6f48bf059  55.737798  12.522954  Home (private)    71191
